## Project Description: Next Word Prediction Using LSTM
#### Project Overview:

This project aims to develop a deep learning model for predicting the next word in a given sequence of words. The model is built using Long Short-Term Memory (LSTM) networks, which are well-suited for sequence prediction tasks. The project includes the following steps:

1- Data Collection: We use the text of Shakespeare's "Hamlet" as our dataset. This rich, complex text provides a good challenge for our model.

2- Data Preprocessing: The text data is tokenized, converted into sequences, and padded to ensure uniform input lengths. The sequences are then split into training and testing sets.

3- Model Building: An LSTM model is constructed with an embedding layer, two LSTM layers, and a dense output layer with a softmax activation function to predict the probability of the next word.

4- Model Training: The model is trained using the prepared sequences, with early stopping implemented to prevent overfitting. Early stopping monitors the validation loss and stops training when the loss stops improving.

5- Model Evaluation: The model is evaluated using a set of example sentences to test its ability to predict the next word accurately.

6- Deployment: A Streamlit web application is developed to allow users to input a sequence of words and get the predicted next word in real-time.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import gutenberg
import pickle


In [2]:
nltk.download("gutenberg")

data = gutenberg.raw("shakespeare-hamlet.txt")

with open("hamlet.txt", "w", encoding="utf-8") as f:
    f.write(data)

with open("hamlet.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()


[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\simi\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [ ]:
### TOKENIZATION
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

total_words = len(tokenizer.word_index) + 1
print("Total words:", total_words)


Total words: 4818


In [ ]:
### Create Input Sequences (n-grams)
input_sequences = []

for line in text.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


In [5]:
### PADDING
max_sequence_len = max(len(seq) for seq in input_sequences)

input_sequences = np.array(
    pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre")
)

print("Input shape:", input_sequences.shape)



Input shape: (25732, 14)


In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

print(X.shape)  # (samples, timesteps)
print(y.shape)  # (samples,)
print(y.dtype)  # int


(25732, 13)
(25732,)
int32


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [29]:
model = Sequential()
model.add(
    Embedding(
        input_dim=total_words,
        output_dim=100,
        input_length=max_sequence_len - 1
    )
)

model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation="softmax"))

from tensorflow.keras.optimizers import Adam


optimizer = Adam(learning_rate=0.001)
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer = optimizer,
    metrics=["accuracy"]
)

model.build(input_shape=(None, max_sequence_len - 1))
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ (None, 13, 100)        │       481,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 13, 150)        │       150,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 13, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ (None, 100)            │       100,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 4818)           │       486,618 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,219,418 (4.65 MB)

 Trainable params: 1,219,418 (4.65 MB)

 Non-trainable params: 0 (0.00 B)

In [35]:
early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


history = model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.0512 - loss: 6.0366 - val_accuracy: 0.0480 - val_loss: 7.1362
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.0551 - loss: 5.8964 - val_accuracy: 0.0511 - val_loss: 7.2344
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0585 - loss: 5.7761 - val_accuracy: 0.0560 - val_loss: 7.3138
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0634 - loss: 5.6602 - val_accuracy: 0.0596 - val_loss: 7.3941
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0656 - loss: 5.5422 - val_accuracy: 0.0604 - val_loss: 7.4844
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0683 - loss: 5.4226 - val_accuracy: 0.0618 - val_loss: 7.5658


In [36]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding="pre"
    )

    predictions = model.predict(token_list, verbose=0)
    predicted_index = np.argmax(predictions, axis=1)[0]

    for word, index in tokenizer.word_index.items():
        if index == predicted_index:
            return word
    return None


In [38]:
input_text = "to be or not to be"
next_word = predict_next_word(
    model, tokenizer, input_text, max_sequence_len
)
print(f"Input text: {input_text}")
print(f"Next word prediction: {next_word}")


Input text: to be or not to be
Next word prediction: lord


In [40]:
np.save("input_sequences.npy", input_sequences)

model.save("next_word_lstm.h5")

with open("tokenizer.pickle", "wb") as f:
    pickle.dump(tokenizer, f)

print("Preprocessing done & saved.")


Preprocessing done & saved.
